In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover,RegexTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler
import shutil
import os
from pyspark.ml.stat import Correlation
import pandas as pd
import seaborn as sns

In [3]:
#spark.stop()

In [4]:
#open Spark Session
spark = SparkSession.builder.appName('prices_houses_v5').master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
#spark.sparkContext.addPyFile("sparkxgb.zip")

In [5]:
#read source I remove last 10 entries for predictions 
data=spark.read.csv("train.csv", inferSchema=True,sep=',',header=True)
datatest=spark.read.csv("test.csv", schema=data.schema,sep=',',header=True)

In [6]:
len(data.columns),len(datatest.columns)

(81, 81)

In [7]:
columnListS = [item[0] for item in data.dtypes if item[1].startswith('string')]
columnListI = [item[0] for item in data.dtypes if item[1].startswith('int')]
columnListS_T = [item[0] for item in datatest.dtypes if item[1].startswith('string')]
columnListI_T = [item[0] for item in datatest.dtypes if item[1].startswith('int')]

In [8]:
datatest=datatest.fillna("NO_Value",subset=columnListS_T)
#datatest=datatest.fillna(0,subset=columnListI_T)
data=data.fillna("NO_Value",subset=columnListS)

In [9]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",handleInvalid="keep").fit(data) for column in list(columnListS) ]
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(data).transform(data)

indexers2 = [StringIndexer(inputCol=column, outputCol=column+"_index",handleInvalid="keep").fit(datatest) for column in list(columnListS_T) ]
pipeline2 = Pipeline(stages=indexers2)
df_rt = pipeline2.fit(datatest).transform(datatest)

In [10]:
columnListS_Cat=[item for item in df_r.columns if str(item).endswith('_index')]

In [11]:
len(columnListS_Cat)+len(columnListI)

81

In [12]:

lr_data = df_r.select(sf.col("SalePrice").alias("label"), *columnListI,*columnListS_Cat).dropna()

columns = lr_data.columns

In [13]:
lr_datat = df_rt.select(sf.col("SalePrice").alias("label"), *columnListI_T,*columnListS_Cat)
columnst = lr_datat.columns

In [14]:
lr_datat=lr_datat.fillna(0)

In [15]:
lr_datat.count()

1459

In [16]:
resumen2=lr_data.toPandas()
resumen2.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
label,1460.0,180921.195890,79442.502883,34900.0,129975.00,163000.0,214000.00,755000.0
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


In [17]:
corr=resumen2.corr()

In [18]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [19]:
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [20]:
features_extra=['OverallQual','YearBuilt','YearRemodAdd','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','GarageCars','GarageArea','ExterQual_index']

In [21]:
type(features_extra)

list

In [22]:
vector = VectorAssembler(inputCols=features_extra, outputCol="features")
scaler = StandardScaler(withMean = True, withStd = True,inputCol="features", outputCol="scaled_features")

stages = [vector, scaler]

pipe = Pipeline(stages=stages)

# we'll be using this data frame
data_out = pipe.fit(lr_data).transform(lr_data).select('id','scaled_features','label')

In [23]:
#lr_datat TEST DATA

vector2 = VectorAssembler(inputCols=features_extra, outputCol="features")
scaler2 = StandardScaler(withMean = True, withStd = True,inputCol="features", outputCol="scaled_features")

stages = [vector2, scaler2]

pipe2 = Pipeline(stages=stages)

# we'll be using this data frame
data_out_t = pipe2.fit(lr_datat).transform(lr_datat).select('id','scaled_features','label')

In [24]:
data_out=data_out.withColumnRenamed('scaled_features','features')

In [25]:
data_out_t=data_out_t.withColumnRenamed('scaled_features','features')

In [26]:
train_data,test_data = data_out.randomSplit([0.7,0.3])

In [27]:
data_out.show()

+---+--------------------+------+
| id|            features| label|
+---+--------------------+------+
|  1|[0.65125609577906...|208500|
|  2|[-0.0718115086600...|181500|
|  3|[0.65125609577906...|223500|
|  4|[0.65125609577906...|140000|
|  5|[1.37432370021817...|250000|
|  6|[-0.7948791130991...|143000|
|  7|[1.37432370021817...|307000|
|  8|[0.65125609577906...|200000|
|  9|[0.65125609577906...|129900|
| 10|[-0.7948791130991...|118000|
| 11|[-0.7948791130991...|129500|
| 12|[2.09739130465727...|345000|
| 13|[-0.7948791130991...|144000|
| 14|[0.65125609577906...|279500|
| 15|[-0.0718115086600...|157000|
| 16|[0.65125609577906...|132000|
| 17|[-0.0718115086600...|149000|
| 18|[-1.5179467175382...| 90000|
| 19|[-0.7948791130991...|159000|
| 20|[-0.7948791130991...|139000|
+---+--------------------+------+
only showing top 20 rows



In [28]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator

In [29]:
lr = LinearRegression()

In [30]:
lrModel = lr.fit(train_data)

In [31]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [27254.1837941,7294.6158053,4532.46591678,4378.90191452,5929.63200309,25916.7516822,-4285.94661948,-352.202186477,9961.71260874,1946.45476459,4999.36429047] Intercept: 180369.76490983402


In [32]:
test_results = lrModel.evaluate(test_data)

In [33]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 36925.34459095824
MSE: 1363481073.1610086
R2: 0.7917329184188171


In [34]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [35]:
gbt = GBTRegressor()

In [36]:
model = gbt.fit(train_data)

In [37]:
predictions = model.transform(test_data)

In [38]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.845549


In [39]:
#model2 = gbt.fit(data_out)

In [40]:
predictions2 = model.transform(data_out_t)

In [41]:
predictions2.count()

1459

In [42]:
predictions2.show(n=1462)

+----+--------------------+-----+------------------+
|  id|            features|label|        prediction|
+----+--------------------+-----+------------------+
|1461|[-0.7316244165748...|    0| 129771.9357290475|
|1462|[-0.0473679910731...|    0| 162746.4014798799|
|1463|[-0.7316244165748...|    0|158432.69923412212|
|1464|[-0.0473679910731...|    0|  190955.336138884|
|1465|[1.32114485993027...|    0| 216960.6427166904|
|1466|[-0.0473679910731...|    0| 184572.1351334654|
|1467|[-0.0473679910731...|    0| 164163.9511906735|
|1468|[-0.0473679910731...|    0|183854.26562116967|
|1469|[0.63688843442858...|    0|179768.17258858477|
|1470|[-1.4158808420764...|    0|110708.29572199218|
|1471|[0.63688843442858...|    0|185976.91853639678|
|1472|[-0.0473679910731...|    0|106251.29204699576|
|1473|[-0.7316244165748...|    0|105353.43840157591|
|1474|[-0.0473679910731...|    0|  190955.336138884|
|1475|[0.63688843442858...|    0|176121.60031101483|
|1476|[2.00540128543196...|    0| 384629.76101

In [43]:
output=predictions2.select('id','prediction')

In [44]:
output.count()

1459

In [45]:
output.write.csv('output6.csv')

In [46]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [75]:
rf = RandomForestRegressor(numTrees=100,maxDepth=18,seed=42,minInfoGain=0.2,subsamplingRate=.9)

In [76]:
modelrf = rf.fit(train_data)

In [77]:
predictionsrf = modelrf.transform(test_data)

In [78]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictionsrf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.863408


In [79]:
predictions3 = modelrf.transform(data_out_t)

In [80]:
output=predictions3.select('id','prediction')

In [81]:
output.write.csv('output7.csv')